In [1]:
import random
import nlpaug.augmenter.word as naw
import string

2024-05-20 09:11:41.756089: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
/Users/florentinafabregas/anaconda3/lib/python3.11/site-packages/transformers/utils/generic.py:260: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(


In [2]:
#Dictionaries for char replacement
approx_dict_lower =  {
 
 'q': ['w', 'a', 's'],
 'w': ['q', 'e', 'a', 's', 'd'],
 'e': ['w', 'r', 's', 'd', 'f'],
 'r': ['e', 't', 'd', 'f', 'g'],
 't': ['r', 'y', 'f', 'g', 'h'],
 'y': ['t', 'u', 'g', 'h', 'j'],
 'u': ['y', 'i', 'h', 'j', 'k'],    
 'i': ['u', 'o', 'j', 'k', 'l'],
 'o': ['i', 'p', 'k', 'l'],
 'p': ['o', 'l'],
 'a': ['q', 'w', 's', 'x', 'z'],
 's': ['e', 'w', 'a', 'd', 'z', 'x'],
 'd': ['r', 'e', 's', 'f', 'x', 'c'],
 'f': ['t', 'r', 'd', 'g', 'c', 'v'],
 'g': ['y', 't', 'f', 'h', 'v', 'b'],
 'h': ['u', 'y', 'g', 'j', 'b', 'n'],
 'j': ['i', 'u', 'h', 'k', 'n', 'm'],
 'k': ['o', 'i', 'j', 'l', 'm', ','],
 'l': ['i', 'p', 'o', 'k', ',', '.'],
 'z': ['x', 'a', 's'],
 'x': ['s', 'd', 'z', 'c'],
 'c': ['d', 'f', 'x', 'v'],
 'v': ['f', 'g', 'c', 'b'],
 'b': ['g', 'h', 'v', 'n'],
 'n': ['h', 'j', 'b', 'm'],
 'm': ['j', 'k', 'n', ','],
 ',': ['k', 'l', 'm', '.'],
 '.': ['l', ',']
    }

approx_dict_upper = {
    'Q': ['W', 'A', 'S'],
 'W': ['Q', 'E', 'A', 'S', 'D'],
 'E': ['W', 'R', 'S', 'D', 'F'],
 'R': ['E', 'T', 'D', 'F', 'G'],
 'T': ['R', 'Y', 'F', 'G', 'H'],
 'Y': ['T', 'U', 'G', 'H', 'J'],
 'U': ['Y', 'I', 'H', 'J', 'K'],
 'I': ['U', 'O', 'J', 'K', 'L'],
 'O': ['I', 'P', 'K', 'L'],
 'P': ['O', 'L'],
 'A': ['Q', 'W', 'S', 'X', 'Z'],
 'S': ['E', 'W', 'A', 'D', 'Z', 'X'],
 'D': ['R', 'E', 'S', 'F', 'X', 'C'],
 'F': ['T', 'R', 'D', 'G', 'C', 'V'],
 'G': ['Y', 'T', 'F', 'H', 'V', 'B'],
 'H': ['U', 'Y', 'G', 'J', 'B', 'N'],
 'J': ['I', 'U', 'H', 'K', 'N', 'M'],
 'K': ['O', 'I', 'J', 'L', 'M', ','],
 'L': ['I', 'P', 'O', 'K', ',', '.'],
 'Z': ['X', 'A', 'S'],
 'X': ['S', 'D', 'Z', 'C'],
 'C': ['D', 'F', 'X', 'V'],
 'V': ['F', 'G', 'C', 'B'],
 'B': ['G', 'H', 'V', 'N'],
 'N': ['H', 'J', 'B', 'M'],
 'M': ['J', 'K', 'N', ','],
 ',': ['K', 'L', 'M', '.'],
 '.': ['L', ',']}

In [4]:
noise_rates = {
    'capitalization_swap': [0, 0.1, 0.2, 0.3, 0.4],  # Adjust the rates as needed
    'character_swap': [0, 0.05, 0.075, 0.1, 0.125, 0.15, 0.17, 0.2],
    'character_removal': [0, 0.05, 0.075, 0.1, 0.125, 0.15, 0.17, 0.2],
    'character_replacement': [0, 0.05, 0.075, 0.1, 0.125, 0.15, 0.17, 0.2]
}

In [6]:
def capitalization_swap(word, rate):
    if random.random() <= rate:
        word = word.capitalize() if word.islower() else word.lower()
    return word

def character_swap(word, rate):
    word_list = list(word)
    word_length = len(word_list)
    swap_count = int(rate * word_length)
    indices = random.sample(range(word_length), swap_count)
    for idx in indices:
        if idx < word_length - 1:
            word_list[idx], word_list[idx + 1] = word_list[idx + 1], word_list[idx]
    return ''.join(word_list)

def character_removal(word, rate):
    word_list = list(word)
    word_length = len(word_list)
    remove_count = int(rate * word_length)
    indices = sorted(random.sample(range(word_length), remove_count), reverse=True)
    for idx in indices:
        del word_list[idx]
    return ''.join(word_list)

def character_replacement(word, rate, approx_dict_lower, approx_dict_upper):
    word_list = list(word)
    word_length = len(word_list)
    replace_count = int(rate * word_length)
    indices = random.sample(range(word_length), replace_count)
    for idx in indices:
        char = word_list[idx]
        if char.islower() and char in approx_dict_lower:
            word_list[idx] = random.choice(approx_dict_lower[char])
        elif char.isupper() and char in approx_dict_upper:
            word_list[idx] = random.choice(approx_dict_upper[char])
    return ''.join(word_list)

In [7]:
def apply_noise_to_word(word, noise_type, rate, approx_dict_lower, approx_dict_upper):
    if noise_type == 'capitalization_swap':
        return capitalization_swap(word, rate)
    elif noise_type == 'character_swap':
        return character_swap(word, rate)
    elif noise_type == 'character_removal':
        return character_removal(word, rate)
    elif noise_type == 'character_replacement':
        return character_replacement(word, rate, approx_dict_lower, approx_dict_upper)
    return word

# Read the input .txt file
with open("data/gold.txt", 'r') as f:
    lines = f.readlines()

# Group the lines into sentences based on empty lines
sentences = []
sentence = []
for line in lines:
    line = line.strip()
    if line:  # If the line is not empty
        sentence.append(line.split('\t'))  # Split into word and tag
    else:  # If the line is empty, indicating end of a sentence
        if sentence:
            sentences.append(sentence)
        sentence = []

# If there are remaining words in the last sentence
if sentence:
    sentences.append(sentence)

# Apply noise functions to the words in each sentence
for noise_type, rates in noise_rates.items():
    for rate in rates:
        altered_sentences = []

        for sentence in sentences:
            altered_sentence = []

            for word, ner_tag in sentence:
                # Apply the corresponding noise function to the word
                altered_word = apply_noise_to_word(word, noise_type, rate, approx_dict_lower, approx_dict_upper)

                # Append the altered word and its NER tag to the sentence
                altered_sentence.append((altered_word, ner_tag))

            # Append the altered sentence to the list of altered sentences
            altered_sentences.append(altered_sentence)

        # Write the altered data to a new .txt file
        output_file = f'data/altered_2/{noise_type}_rate_{rate}.txt'
        with open(output_file, 'w') as f:
            for sentence in altered_sentences:
                for word, ner_tag in sentence:
                    f.write(f'{word}\t{ner_tag}\n')
                f.write('\n')  # Add empty line between sentences
